In [69]:
from datasets import load_dataset

eli5= load_dataset("eli5", split="train_asks[:5000]") 

Found cached dataset eli5 (C:/Users/AIXI/.cache/huggingface/datasets/eli5/LFQA_reddit/1.0.0/17574e5502a10f41bbd17beba83e22475b499fa62caa1384a3d093fc856fe6fa)


In [70]:
eli5= eli5.train_test_split(test_size=0.2)

In [71]:
eli5["train"][22]

{'q_id': '4zt3dd',
 'title': 'If air and water are both considered "fluids" in Physics sense, why are their applications labeled pneumatic/hydraulic, respectfully, and not just hydraulic?',
 'selftext': '',
 'document': '',
 'subreddit': 'askscience',
 'answers': {'a_id': ['d6ykxu3'],
  'text': ["Fluid just means that something flows. The key difference between pneumatics and hydraulics is that while gases are compressible, liquids are not.\n\nThe benefits of hydraulics are that they are more responsive and can supply more power than pneumatics, because the liquids don't compress. However, hydraulics are harder to build and maintain, and are more susceptible to shock damage because of the incompressibility."],
  'score': [20]},
 'title_urls': {'url': []},
 'selftext_urls': {'url': []},
 'answers_urls': {'url': []}}

In [67]:
# eli5= eli5.flatten()
# eli5["train"][22]

In [72]:
import pandas as pd


# Lists to store extracted elements
titles = []
selftexts = []
texts = []

# Loop through each data point
for data in eli5['train']:
    title = data['title']
    selftext = data['selftext']
    text = data['answers']['text'][0]

    titles.append(title)
    selftexts.append(selftext)
    texts.append(text)

# Create a DataFrame
df = pd.DataFrame({
    'Title': titles,
    'Selftext': selftexts,
    'Text': texts
})


In [73]:
df.head()

,Title,Selftext,Text
0,Do humans really affect the environment all th...,"There are the same hurricanes, rainfall, tempe...",[Intergovernmental panel on climate change](_U...
1,What happens to the nucleus in the plasma state?,I read from my teacher's lecture notes that wh...,"No, the nucleus stays intact in a plasma. It t..."
2,reddit 101,Hello /r/AskScience users! This post is part o...,"Very helpful post, thanks for the work it clea..."
3,Why can't we test for neurotransmitter levels ...,It seems as though we have tests available for...,Neurotransmitters generally pop out of one neu...
4,How true is Tesla's earthquake machine story?,,It's plausible. He used some very high power ...


In [74]:
texts= df["Text"].tolist()

In [75]:
from transformers import AutoTokenizer
tokenizer= AutoTokenizer.from_pretrained("distilgpt2")

In [81]:
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

1

In [82]:
# Tokenize the text data
tokenized_texts = tokenizer(texts, truncation=True, padding=True, return_tensors="pt", max_length=128)

In [89]:
print(tokenized_texts['input_ids'])
print(len(tokenized_texts['input_ids']))

tensor([[   58,  9492, 31353,  ...,   338,  4737,   329],
        [ 2949,    11,   262,  ..., 50257, 50257, 50257],
        [16371,  7613,  1281,  ..., 50257, 50257, 50257],
        ...,
        [ 1858,   338,   257,  ..., 50257, 50257, 50257],
        [  464,   411,   513,  ..., 50257, 50257, 50257],
        [ 1639,   423,  3294,  ...,   284,   307,  1498]])
4000


In [76]:
# def preprocess_function(examples, tokenizer):
#     return tokenizer([" ".join(x) for x in examples["answers.text"]])

In [77]:
# tokenized_eli5= eli5.map(
#     preprocess_function(eli5["train"], tokenizer),
#     # preprocess_function,
#     batched=False,
#     # num_proc=4,
#     remove_columns=eli5["train"].column_names,
# )

In [78]:
# block_size = 128


# def group_texts(examples):
#     # Concatenate all texts.
#     concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
#     total_length = len(concatenated_examples[list(examples.keys())[0]])
#     # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
#     # customize this part to your needs.
#     if total_length >= block_size:
#         total_length = (total_length // block_size) * block_size
#     # Split by chunks of block_size.
#     result = {
#         k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
#         for k, t in concatenated_examples.items()
#     }
#     result["labels"] = result["input_ids"].copy()
#     return result

In [79]:
# lm_dataset = tokenized_eli5.map(group_texts, batched=True, num_proc=4)

In [40]:
eli5.flatten()

DatasetDict({
    train: Dataset({
        features: ['q_id', 'title', 'selftext', 'document', 'subreddit', 'answers.a_id', 'answers.text', 'answers.score', 'title_urls.url', 'selftext_urls.url', 'answers_urls.url'],
        num_rows: 4000
    })
    test: Dataset({
        features: ['q_id', 'title', 'selftext', 'document', 'subreddit', 'answers.a_id', 'answers.text', 'answers.score', 'title_urls.url', 'selftext_urls.url', 'answers_urls.url'],
        num_rows: 1000
    })
})